In [1]:
import os
from google.colab import userdata

In [2]:
userdata = userdata.get('github')

os.environ['GITHUB_TOKEN'] = userdata

In [3]:
!git clone https://{userdata}@github.com/miguroi/sistech.git

Cloning into 'sistech'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 2), reused 8 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 292.50 KiB | 2.38 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [4]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [5]:
!ls

sample_data  sistech


In [6]:
cd sistech

/content/sistech


In [7]:
!ls

coursera_courses.csv  README.md  sistech_scraping.ipynb


## Setup

In [8]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

In [9]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [10]:
df = pd.read_csv('coursera_courses.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1750 non-null   object 
 1   organization  1750 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1750 non-null   int64  
 4   difficulty    1750 non-null   object 
 5   course_type   1750 non-null   object 
 6   duration      1750 non-null   object 
 7   skills        1749 non-null   object 
 8   url           1750 non-null   object 
 9   is_free       1750 non-null   bool   
 10  language      0 non-null      float64
 11  course_id     1750 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 152.2+ KB


In [11]:
coursera_df = df[['title', 'skills']].copy()
coursera_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1750 non-null   object
 1   skills  1749 non-null   object
dtypes: object(2)
memory usage: 27.5+ KB


In [12]:
coursera_df.tail()

,title,skills
1745,America's Unwritten Constitution,"Case Law, Court Systems, Legal Proceedings, Legal Research, Policy Analysis, Law, Regulation, and Compliance, Political Sciences, Civil Law, Governance"
1746,Introduction to Ableton Live,"File Management, Musical Composition, Music, Music Theory, Music Performance, Instrumental Music, Automation, Editing, Software Installation, Creativity"
1747,Fixing Healthcare Delivery 2.0: Advanced Lean,"Lean Methodologies, Patient Safety, Patient Flow, Scientific Methods, Health Systems, Health Care Administration, Continuous Quality Improvement (CQI), Health Care, Systems Analysis"
1748,Simulation and modeling of natural processes,"Simulations, Python Programming, Numerical Analysis, Mathematical Modeling, Probability, Artificial Intelligence and Machine Learning (AI/ML), Visualization (Computer Graphics), Mechanics"
1749,Operations and Patient Safety for Healthcare IT Staff,"Telehealth, Patient Safety, Health Technology, Medical Equipment and Technology, Help Desk Support, Technical Support, Desktop Support, Health Care, Issue Tracking, Network Support, Change Control, Technical Documentation, General Networking, Technical Communication"


In [13]:
coursera_df.head()

,title,skills
0,Google Data Analytics,"Data Storytelling, Rmarkdown, Data Literacy, Data Visualization, Data Presentation, Data Ethics, Data Cleansing, Interactive Data Visualization, Data Validation, Ggplot2, Tableau Software, Sampling (Statistics), Presentations, Spreadsheet Software, Data Analysis, Data Visualization Software, Stakeholder Communications, LinkedIn, Interviewing Skills, Applicant Tracking Systems"
1,Google Cybersecurity,"Threat Modeling, Network Security, Incident Response, Vulnerability Management, Computer Security Incident Management, Hardening, Intrusion Detection and Prevention, Cyber Threat Intelligence, Threat Management, Cyber Attacks, Cybersecurity, Network Protocols, Cloud Security, Vulnerability Assessments, Bash (Scripting Language), Debugging, Linux, Interviewing Skills, Python Programming, SQL"
2,Google Project Management:,"Quality Management, Project Management Life Cycle, Requirements Analysis, Project Scoping, Project Closure, Project Management, Project Planning, Agile Project Management, Continuous Improvement Process, Project Controls, Backlogs, Stakeholder Communications, Milestones (Project Management), Quality Assessment, Team Management, Agile Methodology, Project Documentation, Change Management, Interviewing Skills, Applicant Tracking Systems"
3,Google AI Essentials,"Prompt Engineering, Generative AI, Artificial Intelligence and Machine Learning (AI/ML), Large Language Modeling, Process Optimization, Productivity Software, Workforce Development, Digital Transformation, Innovation, Technical Writing, Emerging Technologies, Operational Efficiency, Business Solutions, Machine Learning Software, Data Security, Critical Thinking, Analysis, Data Analysis, Data Quality"
4,Google Digital Marketing & E-commerce,"Data Storytelling, Search Engine Marketing, Media Planning, Social Media Marketing, Google Ads, Email Marketing, Social Media Strategy, Search Engine Optimization, Order Fulfillment, Social Media Management, Performance Measurement, Spreadsheet Software, A/B Testing, Customer Retention, E-Commerce, Campaign Management, Loyalty Programs, Marketing, Interviewing Skills, Applicant Tracking Systems"


## Preprocessing

### Lowercasing

In [14]:
def lowercasing(text):
    return text.lower()

coursera_df['title'] = coursera_df['title'].apply(lowercasing)
coursera_df['skills'] = coursera_df['skills'].astype(str).apply(lowercasing)

coursera_df.head()

,title,skills
0,google data analytics,"data storytelling, rmarkdown, data literacy, data visualization, data presentation, data ethics, data cleansing, interactive data visualization, data validation, ggplot2, tableau software, sampling (statistics), presentations, spreadsheet software, data analysis, data visualization software, stakeholder communications, linkedin, interviewing skills, applicant tracking systems"
1,google cybersecurity,"threat modeling, network security, incident response, vulnerability management, computer security incident management, hardening, intrusion detection and prevention, cyber threat intelligence, threat management, cyber attacks, cybersecurity, network protocols, cloud security, vulnerability assessments, bash (scripting language), debugging, linux, interviewing skills, python programming, sql"
2,google project management:,"quality management, project management life cycle, requirements analysis, project scoping, project closure, project management, project planning, agile project management, continuous improvement process, project controls, backlogs, stakeholder communications, milestones (project management), quality assessment, team management, agile methodology, project documentation, change management, interviewing skills, applicant tracking systems"
3,google ai essentials,"prompt engineering, generative ai, artificial intelligence and machine learning (ai/ml), large language modeling, process optimization, productivity software, workforce development, digital transformation, innovation, technical writing, emerging technologies, operational efficiency, business solutions, machine learning software, data security, critical thinking, analysis, data analysis, data quality"
4,google digital marketing & e-commerce,"data storytelling, search engine marketing, media planning, social media marketing, google ads, email marketing, social media strategy, search engine optimization, order fulfillment, social media management, performance measurement, spreadsheet software, a/b testing, customer retention, e-commerce, campaign management, loyalty programs, marketing, interviewing skills, applicant tracking systems"


In [15]:
df.tail()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,url,is_free,language,course_id
1745,America's Unwritten Constitution,Yale University,4.812950,278,Mixed,Course,THREE_TO_SIX_MONTHS,"Case Law, Court Systems, Legal Proceedings, Legal Research, Policy Analysis, Law, Regulation, and Compliance, Political Sciences, Civil Law, Governance",/learn/unwritten-constitution,True,NaN,course~aAo9tS8NEeWv_w7cMMH1Uw
1746,Introduction to Ableton Live,Berklee,4.775382,1113,Mixed,Course,ONE_TO_FOUR_WEEKS,"File Management, Musical Composition, Music, Music Theory, Music Performance, Instrumental Music, Automation, Editing, Software Installation, Creativity",/learn/ableton-live,False,NaN,course~-nRRAPmuEeSndSIAC7LTLQ
1747,Fixing Healthcare Delivery 2.0: Advanced Lean,University of Florida,4.727273,44,Mixed,Course,ONE_TO_THREE_MONTHS,"Lean Methodologies, Patient Safety, Patient Flow, Scientific Methods, Health Systems, Health Care Administration, Continuous Quality Improvement (CQI), Health Care, Systems Analysis",/learn/fixing-healthcare-delivery-advanced-lean,False,NaN,course~7XFCjCE3EeeIYhLE7kr1cg
1748,Simulation and modeling of natural processes,University of Geneva,4.275000,400,Mixed,Course,ONE_TO_THREE_MONTHS,"Simulations, Python Programming, Numerical Analysis, Mathematical Modeling, Probability, Artificial Intelligence and Machine Learning (AI/ML), Visualization (Computer Graphics), Mechanics",/learn/modeling-simulation-natural-processes,True,NaN,course~qwwhTCF_EeWXzxJxfIL00w
1749,Operations and Patient Safety for Healthcare IT Staff,Johns Hopkins University,4.844560,193,Mixed,Course,ONE_TO_FOUR_WEEKS,"Telehealth, Patient Safety, Health Technology, Medical Equipment and Technology, Help Desk Support, Technical Support, Desktop Support, Health Care, Issue Tracking, Network Support, Change Control, Technical Documentation, General Networking, Technical Communication",/learn/healthcare-it-operations-patient-safety,False,NaN,course~z7bOth-eEeuQ9xLSu4r6yQ


### Noise Removal

In [16]:
import re

def clean_noise(text):
    text = str(text)

    text = re.sub(r'<.*?>', '', text)

    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    text = re.sub(r'#\w+', '', text)

    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'\d+', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

coursera_df['title'] = coursera_df['title'].apply(clean_noise)
coursera_df['skills'] = coursera_df['skills'].apply(clean_noise)

coursera_df.head()

,title,skills
0,google data analytics,data storytelling rmarkdown data literacy data visualization data presentation data ethics data cleansing interactive data visualization data validation ggplot tableau software sampling statistics presentations spreadsheet software data analysis data visualization software stakeholder communications linkedin interviewing skills applicant tracking systems
1,google cybersecurity,threat modeling network security incident response vulnerability management computer security incident management hardening intrusion detection and prevention cyber threat intelligence threat management cyber attacks cybersecurity network protocols cloud security vulnerability assessments bash scripting language debugging linux interviewing skills python programming sql
2,google project management,quality management project management life cycle requirements analysis project scoping project closure project management project planning agile project management continuous improvement process project controls backlogs stakeholder communications milestones project management quality assessment team management agile methodology project documentation change management interviewing skills applicant tracking systems
3,google ai essentials,prompt engineering generative ai artificial intelligence and machine learning aiml large language modeling process optimization productivity software workforce development digital transformation innovation technical writing emerging technologies operational efficiency business solutions machine learning software data security critical thinking analysis data analysis data quality
4,google digital marketing ecommerce,data storytelling search engine marketing media planning social media marketing google ads email marketing social media strategy search engine optimization order fulfillment social media management performance measurement spreadsheet software ab testing customer retention ecommerce campaign management loyalty programs marketing interviewing skills applicant tracking systems


In [17]:
coursera_df.tail()

,title,skills
1745,americas unwritten constitution,case law court systems legal proceedings legal research policy analysis law regulation and compliance political sciences civil law governance
1746,introduction to ableton live,file management musical composition music music theory music performance instrumental music automation editing software installation creativity
1747,fixing healthcare delivery advanced lean,lean methodologies patient safety patient flow scientific methods health systems health care administration continuous quality improvement cqi health care systems analysis
1748,simulation and modeling of natural processes,simulations python programming numerical analysis mathematical modeling probability artificial intelligence and machine learning aiml visualization computer graphics mechanics
1749,operations and patient safety for healthcare it staff,telehealth patient safety health technology medical equipment and technology help desk support technical support desktop support health care issue tracking network support change control technical documentation general networking technical communication


### Compound Handling

In [18]:
import spacy
from collections import Counter, defaultdict

In [19]:
def detect_compound_patterns(df, text_column, min_freq=3):
    """Detect compounds using POS patterns and frequency"""

    nlp = spacy.load("en_core_web_sm")

    compound_patterns = [
        ['NOUN', 'NOUN'],           # "data science"
        ['ADJ', 'NOUN'],            # "machine learning"
        ['NOUN', 'NOUN', 'NOUN'],   # "natural language processing"
        ['ADJ', 'NOUN', 'NOUN'],    # "artificial neural networks"
        ['NOUN', 'ADP', 'NOUN'],    # "introduction to python" (but filter common prepositions)
        ['VERB', 'NOUN'],           # "deep learning" (when deep is tagged as VERB)
    ]

    detected_sequences = defaultdict(int)
    texts = df[text_column].fillna('').astype(str)

    for doc in nlp.pipe(texts, batch_size=50):
        tokens = [token for token in doc if token.is_alpha and not token.is_space]

        # Check each pattern
        for pattern in compound_patterns:
            pattern_length = len(pattern)

            for i in range(len(tokens) - pattern_length + 1):
                sequence = tokens[i:i + pattern_length]

                # Check if POS pattern matches
                if [token.pos_ for token in sequence] == pattern:

                    # Additional filters
                    if all(not token.is_stop or token.pos_ == 'ADP' for token in sequence):

                        # Special handling for prepositions
                        if 'ADP' in pattern:
                            if sequence[1].text.lower() in ['to', 'of', 'for', 'in']:
                                phrase = ' '.join([token.text.lower() for token in sequence])
                                detected_sequences[phrase] += 1
                        else:
                            phrase = ' '.join([token.text.lower() for token in sequence])
                            detected_sequences[phrase] += 1

    # Filter by frequency and convert to underscore format
    compounds = [phrase.replace(' ', '_')
                for phrase, count in detected_sequences.items()
                if count >= min_freq]

    return compounds, detected_sequences

compounds, sequences = detect_compound_patterns(coursera_df, 'skills', min_freq=5)
print("Pattern-detected compounds:", compounds[:15])

Pattern-detected compounds: ['data_literacy', 'literacy_data', 'data_visualization', 'visualization_data', 'data_presentation', 'presentation_data', 'data_ethics', 'ethics_data', 'data_validation', 'tableau_software', 'sampling_statistics', 'spreadsheet_software', 'software_data', 'data_analysis', 'analysis_data']


In [20]:
def preserve_compounds(text):
      if pd.isna(text):
          return text
      text_modified = str(text).lower()
      for compound in sorted(compounds, key=len, reverse=True):
          original = compound.replace('_', ' ')
          if original in text_modified:
              text_modified = text_modified.replace(original, compound)
      return text_modified

In [21]:
coursera_df['title'] = coursera_df['title'].apply(preserve_compounds)
coursera_df['skills'] = coursera_df['skills'].apply(preserve_compounds)

In [22]:
coursera_df.head()

,title,skills
0,google data analytics,data storytelling rmarkdown data_literacy_data_visualization_data_presentation_data_ethics_data_cleansing interactive_data_visualization_data_validation ggplot tableau_software sampling_statistics presentations spreadsheet_software_data_analysis_data_visualization_software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,threat_modeling network_security_incident_response vulnerability_management computer_security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence_threat_management_cyber_attacks cybersecurity_network_protocols cloud_security_vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project_management,quality_management_project_management_life_cycle requirements_analysis_project scoping_project_closure project_management_project_planning agile_project_management continuous_improvement_process_project controls backlogs stakeholder_communications milestones_project_management_quality assessment team_management agile_methodology_project_documentation change_management interviewing_skills applicant_tracking_systems
3,google ai essentials,prompt_engineering_generative ai artificial_intelligence and machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solutions machine_learning_software_data_security critical_thinking analysis_data_analysis_data_quality
4,google digital_marketing ecommerce,data storytelling search_engine_marketing media planning social_media_marketing google ads email_marketing social_media_strategy search_engine_optimization order fulfillment social_media_management_performance_measurement spreadsheet_software ab testing customer_retention ecommerce campaign_management loyalty programs marketing interviewing_skills applicant_tracking_systems


### Stopwords Removal

In [23]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [24]:
def remove_stopwords(text):
    text = str(text).lower()

    preserve_terms = {
        'ai', 'ml', 'ar', 'vr', 'ui', 'ux', 'api', 'aws', 'gcp', 'sql',
        'html', 'css', 'ios', 'android', 'react', 'node', 'vue', 'php',
        'java', 'python', 'r', 'scala', 'go', 'ruby', 'swift', 'kotlin',
        '3d', 'bi', 'etl', 'devops', 'cicd', 'ci', 'cd', 'qa', 'nlp',
        'cnn', 'rnn', 'gan', 'bert', 'gpt', 'llm', 'iot', 'erp', 'crm'
    }

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))

    filtered_tokens = []
    for token in tokens:
        if token in preserve_terms or '_' in token:
            filtered_tokens.append(token)
        elif token not in stop_words and token.isalpha():
            if len(token) > 2:
                filtered_tokens.append(token)

    return ' '.join(filtered_tokens)

In [25]:
coursera_df.tail()

,title,skills
1745,americas unwritten constitution,case_law court_systems legal_proceedings legal_research policy_analysis law_regulation and compliance political_sciences civil_law governance
1746,introduction to ableton live,file_management musical_composition music music_theory music_performance instrumental_music automation editing software_installation creativity
1747,fixing healthcare delivery advanced lean,lean_methodologies patient_safety patient flow scientific_methods health_systems_health_care administration continuous_quality_improvement cqi health_care systems_analysis
1748,simulation and_modeling of natural processes,simulations python_programming numerical_analysis mathematical_modeling_probability artificial_intelligence and machine_learning_aiml visualization_computer_graphics mechanics
1749,operations and patient_safety for healthcare it staff,telehealth_patient_safety health_technology medical_equipment and technology help desk support technical_support desktop_support health_care issue tracking network support change_control technical_documentation general networking technical_communication


In [26]:
coursera_df.head(100)

,title,skills
0,google data analytics,data storytelling rmarkdown data_literacy_data_visualization_data_presentation_data_ethics_data_cleansing interactive_data_visualization_data_validation ggplot tableau_software sampling_statistics presentations spreadsheet_software_data_analysis_data_visualization_software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,threat_modeling network_security_incident_response vulnerability_management computer_security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence_threat_management_cyber_attacks cybersecurity_network_protocols cloud_security_vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project_management,quality_management_project_management_life_cycle requirements_analysis_project scoping_project_closure project_management_project_planning agile_project_management continuous_improvement_process_project controls backlogs stakeholder_communications milestones_project_management_quality assessment team_management agile_methodology_project_documentation change_management interviewing_skills applicant_tracking_systems
3,google ai essentials,prompt_engineering_generative ai artificial_intelligence and machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solutions machine_learning_software_data_security critical_thinking analysis_data_analysis_data_quality
4,google digital_marketing ecommerce,data storytelling search_engine_marketing media planning social_media_marketing google ads email_marketing social_media_strategy search_engine_optimization order fulfillment social_media_management_performance_measurement spreadsheet_software ab testing customer_retention ecommerce campaign_management loyalty programs marketing interviewing_skills applicant_tracking_systems
5,google it support,computer networking package and software_management_network_troubleshooting systems administration cloud_infrastructure it infrastructure information_systems_security tcpip network_security file_systems microsoft windows computer_security_network_architecture application_security desktop_support computer_hardware technical_support interviewing_skills applicant_tracking_systems professional_development
6,google ux design,responsive_web_design storyboarding wireframing user_experience_design uiux research_usability_testing information_architecture presentations user_research figma design_software_design_reviews persona_user_experience web_content_accessibility_guidelines mobile_development_user_story cross platform development_data_ethics usability interviewing_skills applicant_tracking_systems
7,machine_learning,unsupervised_learning supervised_learning_machine_learning_methods classification and regression_tree_cart artificial_intelligence and machine_learning_aiml applied_machine_learning_machine_learning_algorithms_machine_learning jupyter data_ethics decision_tree learning_tensorflow scikit learn_machine_learning_library artificial_intelligence numpy predictive_modeling deep_learning_reinforcement_learning random_forest algorithm feature_engineering
8,google prompting essentials,data_visualization prompt_engineering_data_presentation graphing generative ai_agents chatgpt ai_personalization generative ai timelines data_analysis taking_meeting minutes artificial_intelligence complex_problem solving ideation business_correspondence report writing solution_design critical_thinking document_management_machine_learning
9,ai for everyone,data_ethics market_opportunities artificial_intelligence and machine_learning_aiml artificial_intelligence team building machine_learning strategic_thinking data_science needs_assessment deep_learning business_ethics artificial_neural_networks engineering_manage

### Lemmatization

In [27]:
nltk.download('wordnet')
nltk.download('omw-1.4')

def apply_lemmatization(text):
    text = str(text)
    lemmatizer = WordNetLemmatizer()

    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(lemmatized_words)

coursera_df['title'] = coursera_df['title'].apply(apply_lemmatization)
coursera_df['skills'] = coursera_df['skills'].apply(apply_lemmatization)

coursera_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,title,skills
0,google data analytics,data storytelling rmarkdown data_literacy_data_visualization_data_presentation_data_ethics_data_cleansing interactive_data_visualization_data_validation ggplot tableau_software sampling_statistics presentation spreadsheet_software_data_analysis_data_visualization_software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,threat_modeling network_security_incident_response vulnerability_management computer_security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence_threat_management_cyber_attacks cybersecurity_network_protocols cloud_security_vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project_management,quality_management_project_management_life_cycle requirements_analysis_project scoping_project_closure project_management_project_planning agile_project_management continuous_improvement_process_project control backlog stakeholder_communications milestones_project_management_quality assessment team_management agile_methodology_project_documentation change_management interviewing_skills applicant_tracking_systems
3,google ai essential,prompt_engineering_generative ai artificial_intelligence and machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solution machine_learning_software_data_security critical_thinking analysis_data_analysis_data_quality
4,google digital_marketing ecommerce,data storytelling search_engine_marketing medium planning social_media_marketing google ad email_marketing social_media_strategy search_engine_optimization order fulfillment social_media_management_performance_measurement spreadsheet_software ab testing customer_retention ecommerce campaign_management loyalty program marketing interviewing_skills applicant_tracking_systems


In [28]:
coursera_df.tail(100)

,title,skills
1650,initiating and planning_projects,stakeholder_management_conflict_management_project_management_project_planning_project_management_life_cycle_project_documentation_project scoping_project_coordination organizational_structure planning influencing
1651,web and blockchain fundamental,digital_assets blockchain emerging_technologies fintech business_transformation digital_transformation ecommerce technical_design scalability business_strategy governance
1652,philosophy and the science introduction to the philosophy of cognitive science,artificial_intelligence agentic_systems psychology anthropology human learning human_development science and research
1653,managing the company of the future,organizational_change leadership and management_business_management_innovation organizational_structure leadership personal_development goal setting
1654,hypothesisdriven_development,usability_testing continuous_delivery product_testing agile_product_development devops agile_methodology user_research prototyping product_development lean_methodologies innovation
1655,microeconomics when market fail,market_dynamics economics socioeconomics supply and demand market_analysis_market share competitive_analysis public_policies policty analysis_research and development consumer_behaviour
1656,introduction to human behavioral genetics,mental and behavioral_health social_science psychology psychiatry biostatistics medical_science and research molecular_biology research_methodologies biology
1657,finance for manager,financial_statement_analysis operational_analysis working_capital balance_sheet management accounting_income_statement financial_forecasting forecasting return on investment financial_analysis_cash_flows cash_flow forecasting finance budget_management_risk_analysis financial_planning account receivable business_analysis inventory_management_system growth_strategies
1658,resilience in child exposed to trauma disaster and war global perspective,child welfare crisis intervention research_research_methodologies working with child trauma care child_development emergency_response risk_analysis social_science human_development international_relations cultural_diversity system thinking culture
1659,oceanography a key to better understand our world,physical_science water_resources geographic_information_systems geospatial_information and technology chemistry environment environmental_science physic biology remote_access_systems


In [29]:
coursera_df.head(1000)

,title,skills
0,google data analytics,data storytelling rmarkdown data_literacy_data_visualization_data_presentation_data_ethics_data_cleansing interactive_data_visualization_data_validation ggplot tableau_software sampling_statistics presentation spreadsheet_software_data_analysis_data_visualization_software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,threat_modeling network_security_incident_response vulnerability_management computer_security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence_threat_management_cyber_attacks cybersecurity_network_protocols cloud_security_vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project_management,quality_management_project_management_life_cycle requirements_analysis_project scoping_project_closure project_management_project_planning agile_project_management continuous_improvement_process_project control backlog stakeholder_communications milestones_project_management_quality assessment team_management agile_methodology_project_documentation change_management interviewing_skills applicant_tracking_systems
3,google ai essential,prompt_engineering_generative ai artificial_intelligence and machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solution machine_learning_software_data_security critical_thinking analysis_data_analysis_data_quality
4,google digital_marketing ecommerce,data storytelling search_engine_marketing medium planning social_media_marketing google ad email_marketing social_media_strategy search_engine_optimization order fulfillment social_media_management_performance_measurement spreadsheet_software ab testing customer_retention ecommerce campaign_management loyalty program marketing interviewing_skills applicant_tracking_systems
5,google it support,computer networking package and software_management_network_troubleshooting system administration cloud_infrastructure it infrastructure information_systems_security tcpip network_security file_system microsoft window computer_security_network_architecture application_security desktop_support computer_hardware technical_support interviewing_skills applicant_tracking_systems professional_development
6,google ux design,responsive_web_design storyboarding wireframing user_experience_design uiux research_usability_testing information_architecture presentation user_research figma design_software_design_reviews persona_user_experience web_content_accessibility_guidelines mobile_development_user_story cross platform development_data_ethics usability interviewing_skills applicant_tracking_systems
7,machine_learning,unsupervised_learning supervised_learning_machine_learning_methods classification and regression_tree_cart artificial_intelligence and machine_learning_aiml applied_machine_learning_machine_learning_algorithms_machine_learning jupyter data_ethics decision_tree learning_tensorflow scikit learn_machine_learning_library artificial_intelligence numpy predictive_modeling deep_learning_reinforcement_learning random_forest algorithm feature_engineering
8,google prompting essential,data_visualization prompt_engineering_data_presentation graphing generative ai_agents chatgpt ai_personalization generative ai timeline data_analysis taking_meeting minute artificial_intelligence complex_problem solving ideation business_correspondence report writing solution_design critical_thinking document_management_machine_learning
9,ai for everyone,data_ethics market_opportunities artificial_intelligence and machine_learning_aiml artificial_intelligence team building machine_learning strategic_thinking data_science needs_assessment deep_learning business_ethics artificial_neural_networks engineering_management


In [30]:
coursera_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1750 non-null   object
 1   skills  1750 non-null   object
dtypes: object(2)
memory usage: 27.5+ KB


### Pipeline

In [47]:
df_coursera = df.copy()

In [48]:
df_coursera = df_coursera[['title', 'skills']].copy()
df_coursera.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1750 non-null   object
 1   skills  1749 non-null   object
dtypes: object(2)
memory usage: 27.5+ KB


In [49]:
df_coursera.dropna(inplace=True)
df_coursera.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1749 entries, 0 to 1749
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1749 non-null   object
 1   skills  1749 non-null   object
dtypes: object(2)
memory usage: 41.0+ KB


In [50]:
def cleaning_pipeline(text):
    text = lowercasing(text)
    text = clean_noise(text)
    text = preserve_compounds(text)
    text = remove_stopwords(text)
    text = apply_lemmatization(text)
    return text

df_coursera['title'] = df_coursera['title'].apply(cleaning_pipeline)
df_coursera['skills'] = df_coursera['skills'].apply(cleaning_pipeline)

df_coursera.head()

,title,skills
0,google data analytics,data storytelling rmarkdown data_literacy_data_visualization_data_presentation_data_ethics_data_cleansing interactive_data_visualization_data_validation ggplot tableau_software sampling_statistics presentation spreadsheet_software_data_analysis_data_visualization_software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,threat_modeling network_security_incident_response vulnerability_management computer_security_incident_management hardening intrusion_detection prevention cyber_threat_intelligence_threat_management_cyber_attacks cybersecurity_network_protocols cloud_security_vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project_management,quality_management_project_management_life_cycle requirements_analysis_project scoping_project_closure project_management_project_planning agile_project_management continuous_improvement_process_project control backlog stakeholder_communications milestones_project_management_quality assessment team_management agile_methodology_project_documentation change_management interviewing_skills applicant_tracking_systems
3,google ai essential,prompt_engineering_generative ai artificial_intelligence machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solution machine_learning_software_data_security critical_thinking analysis_data_analysis_data_quality
4,google digital_marketing ecommerce,data storytelling search_engine_marketing medium planning social_media_marketing google ad email_marketing social_media_strategy search_engine_optimization order fulfillment social_media_management_performance_measurement spreadsheet_software testing customer_retention ecommerce campaign_management loyalty program marketing interviewing_skills applicant_tracking_systems


In [51]:
df_coursera.tail()

,title,skills
1745,america unwritten constitution,case_law court_systems legal_proceeding legal_research policy_analysis law_regulation compliance political_science civil_law governance
1746,introduction ableton live,file_management musical_composition music music_theory music_performance instrumental_music automation editing software_installation creativity
1747,fixing healthcare delivery advanced lean,lean_methodologies patient_safety patient flow scientific_method health_systems_health_care administration continuous_quality_improvement cqi health_care systems_analysis
1748,simulation and_modeling natural process,simulation python_programming numerical_analysis mathematical_modeling_probability artificial_intelligence machine_learning_aiml visualization_computer_graphics mechanic
1749,operation patient_safety healthcare staff,telehealth_patient_safety health_technology medical_equipment technology help desk support technical_support desktop_support health_care issue tracking network support change_control technical_documentation general networking technical_communication


## Final Dataset

In [55]:
df_coursera_csv = 'coursera_courses_cleaned.csv'
df_coursera.to_csv(df_coursera_csv, index=False)

print(f"Data successfully saved to file: '{df_coursera_csv}'")

Data successfully saved to file: 'coursera_courses_cleaned.csv'


In [56]:
df_coursera_json = 'coursera_courses_cleaned.json'
df_coursera.to_json(df_coursera_json, orient='records', indent=4)

print(f"Data successfully saved to file: '{df_coursera_json}'")

Data successfully saved to file: 'coursera_courses_cleaned.json'
